In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from classifiers import LogisticRegression as LogReg
from classifiers import PyLogisticRegression as PyLogReg

In [3]:
df = pd.read_csv('house_data.csv')

In [4]:
df.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade,yr_built
0,221900.0,3,1.00,1180,5650,1.0,3,7,1955
1,538000.0,3,2.25,2570,7242,2.0,3,7,1951
2,180000.0,2,1.00,770,10000,1.0,3,6,1933
3,604000.0,4,3.00,1960,5000,1.0,5,7,1965
4,510000.0,3,2.00,1680,8080,1.0,3,8,1987


In [5]:
y = (df['price'] > df['price'].mean()).astype('float64')
X = df.loc[:,'bedrooms':].astype('float64')

In [6]:
logit = PyLogReg(num_iter=100, fit_intercept=False)

In [7]:
logit.fit(X.values.astype('float64'), y.values.astype('float64'))

/home/oupton/git/CUDA_Stats/logreg/classifiers.py:19: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


In [8]:
logit.theta

array([-1.33800637e-01,  4.09142849e-03,  6.57718073e+01,  3.35666405e+01,
       -1.45910210e-02, -2.16409010e-01, -2.01157337e-01, -1.28157098e+02])

In [10]:
np.savetxt('theta.csv', logit.theta, delimiter=',')

In [12]:
yt = logit.predict_prob(X)

In [13]:
np.savetxt('yt.csv', yt, delimiter=',')

In [11]:
logit.fit(X.values.astype('float64'), y.values.astype('float64'))

In [12]:
logit.theta

array([-1.7734364 ,  0.2462082 , -1.22313193,  0.64911671, -7.33802003,
        1.43797842, -2.12991027,  0.46665493])

In [13]:
import ctypes
from ctypes import *
ctypes.sizeof(c_double)

8